In [7]:
import os
import re
import pandas as pd
from collections import defaultdict

# Pfad zum Hauptverzeichnis, in dem die Unterverzeichnisse liegen
BASE_DIR = "data/liboqs"

# Funktion zur Extraktion des Modelnamens
def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_liboqs_(.*?)_\d{4}\.\d{2}\.\d{2}_\d{2}\.\d{2}", directory_name)
    return match.group(1) if match else directory_name

# Funktion zur Verarbeitung der Signature-Speed-Logs
def parse_liboqs_sig_speed_log(log_path, model_name):
    """Liest die liboqs Signature log-Datei und gibt eine Liste mit Ergebnissen zurück."""
    if not os.path.isfile(log_path):
        return []
    
    with open(log_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    data = []
    current_algorithm = None
    
    for line in lines:
        parts = [p.strip() for p in line.split("|")]
        if len(parts) < 7:
            continue
        
        op, iters_str, _, _, _, highprec_str, _ = parts
        
        if op and not iters_str:
            current_algorithm = op  # Neuer Algorithmus erkannt
        elif op in ("keypair", "sign", "verify") and iters_str.isdigit() and highprec_str.isdigit():
            rounds = int(iters_str)
            ms_mean = round(float(highprec_str) / 1_000_000.0, 3)  # Umrechnung ns -> ms
            data.append((model_name, current_algorithm, op, rounds, ms_mean))
    
    # Gruppierung nach Modell, Algorithmus und Operation
    grouped = defaultdict(dict)
    for m, alg, op, rnd, ms in data:
        grouped[(m, alg)][op] = (rnd, ms)
    
    # Erstellung der Datensätze
    records = []
    for (m, alg), ops_dict in grouped.items():
        if all(k in ops_dict for k in ["keypair", "sign", "verify"]):
            k_r, k_ms = ops_dict["keypair"]
            s_r, s_ms = ops_dict["sign"]
            v_r, v_ms = ops_dict["verify"]
            
            total_ms = round(k_ms + s_ms + v_ms, 3)
            
            records.append({
                "Model": m,
                "Algorithmus": alg,
                "Runden_keypair": k_r,
                "keypair (ms)": k_ms,
                "Runden_sign": s_r,
                "sign (ms)": s_ms,
                "Runden_verify": v_r,
                "verify (ms)": v_ms,
                "Total Time (ms)": total_ms
            })
    
    return records

# Verarbeitung aller Verzeichnisse
all_records = []
for directory in os.listdir(BASE_DIR):
    full_path = os.path.join(BASE_DIR, directory)
    if os.path.isdir(full_path):
        log_file = os.path.join(full_path, "liboqs_sig_speed.log")
        if os.path.isfile(log_file):
            model_name = extract_model_name(directory)
            recs = parse_liboqs_sig_speed_log(log_file, model_name)
            all_records.extend(recs)

# DataFrame erstellen
df = pd.DataFrame(all_records)
df = df[["Model", "Algorithmus", "Runden_keypair", "keypair (ms)", "Runden_sign", "sign (ms)", "Runden_verify", "verify (ms)", "Total Time (ms)"]]
df = df.sort_values(by=["Algorithmus", "Model"])

# Speichern als CSV
df.to_csv("output/csv/liboqs_time_sig.csv", index=False)

# Formatierte Ausgabe für Jupyter Notebook
df_formatted = df.copy()
df_formatted[df_formatted.select_dtypes(include=['float64', 'float32']).columns] = (
    df_formatted.select_dtypes(include=['float64', 'float32']).map(lambda x: f"{x:.3f}".replace('.', ','))
)
df_formatted

,Model,Algorithmus,Runden_keypair,keypair (ms),Runden_sign,sign (ms),Runden_verify,verify (ms),Total Time (ms)
47,Pi1b,Dilithium2,168,"17,946",75,"40,132",175,"17,134","75,212"
0,Pi1bPlus,Dilithium2,177,"16,972",74,"40,761",185,"16,291","74,024"
94,Pi3b,Dilithium2,3832,"0,783",1339,"2,241",4084,"0,734","3,758"
141,Pi4b,Dilithium2,17462,"0,172",6262,"0,479",19637,"0,153","0,804"
188,PiBanana,Dilithium2,1270,"2,363",463,"6,513",1303,"2,303","11,179"
...,...,...,...,...,...,...,...,...,...
46,Pi1bPlus,cross-rsdpg-256-small,427,"7,027",2,"1514,304",4,"925,974","2447,305"
140,Pi3b,cross-rsdpg-256-small,6803,"0,441",21,"145,386",28,"108,744","254,571"
187,Pi4b,cross-rsdpg-256-small,30609,"0,098",113,"26,622",165,"18,204","44,924"
234,PiBanana,cross-rsdpg-256-small,3211,"0,934",13,"242,824",21,"147,617","391,375"
